In [3]:
import re
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

ModuleNotFoundError: No module named 'numpy'

In [12]:
df = pd.read_csv("Historical Data/AMZN")
df

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30 00:00:00,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [13]:
def get_stock(symbol):
  stock = df[df['symbol']==symbol]
  stock_prices = stock.close.values.astype('float32')
  stock_prices = stock_prices.reshape(len(stock_prices), 1)
  return stock_prices

yahoo_stock = get_stock('YHOO')
yahoo_stock

array([[17.1 ],
       [17.23],
       [17.17],
       ...,
       [38.73],
       [38.64],
       [38.67]], dtype=float32)

In [14]:
def normalization(x):
  return (x-min(x)/max(x)-min(x))
yahoo_stock_n = normalization(yahoo_stock)

In [15]:
def train_test_split(stock):
  train_size = int(len(stock) * 0.80)
  test_size = len(stock) - train_size
  train, test = stock[0:train_size,:], stock[train_size:len(stock),:]
  return train, test


In [19]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
def create_trainTest():
	train, test = train_test_split(yahoo_stock)
	# reshape into X=t and Y=t+1
	look_back = 1
	trainX, trainY = create_dataset(train, look_back)
	testX, testY = create_dataset(test, look_back)
	trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
	testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
	return trainX, testX, trainY, testY

In [ ]:
def new_trainTest():
    """
    This function will flow in the new data and preprocess it
    """
    return None

In [21]:
from tensorflow.keras import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

def first_train():
  trainX, testX, trainY, testY = create_trainTest()
  model = Sequential()
  model.add(LSTM(units=50,return_sequences=True,input_shape=(trainX.shape[1], 1)))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50,return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50,return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))
  model.compile(optimizer='adam',loss='mean_squared_error')
  model.fit(trainX, trainY,epochs=15,batch_size=32)
  results = model.evaluate(testX, testY, batch_size=32)
  print("test loss, test MSE:", results)
  return model
  

def re_train(model):
  new_trainX, new_trainY, new_testX, new_testY = new_trainTest()
  model.save('test1.h5')
  model1 = tf.keras.models.load_model('test1.h5')
  model = Sequential([model1])
  model.compile(optimizer='adam',loss='mean_squared_error') 
  model.fit(new_trainX, new_trainY, epochs = 15, batch_size=32)
  results = model.evaluate(new_testX, new_testY, batch_size=32)
  print("test loss, test MSE:", results)

def main():
  model = first_train()
  for i in range(0,5):
    re_train(model)
main()

Epoch 1/15
44/44 [==============================] - 3s 3ms/step - loss: 735.5706
Epoch 2/15
44/44 [==============================] - 0s 3ms/step - loss: 446.9911
Epoch 3/15
44/44 [==============================] - 0s 3ms/step - loss: 305.9675
Epoch 4/15
44/44 [==============================] - 0s 3ms/step - loss: 256.2813
Epoch 5/15
44/44 [==============================] - 0s 3ms/step - loss: 223.5452
Epoch 6/15
44/44 [==============================] - 0s 3ms/step - loss: 198.0686
Epoch 7/15
44/44 [==============================] - 0s 3ms/step - loss: 178.0656
Epoch 8/15
44/44 [==============================] - 0s 3ms/step - loss: 165.6948
Epoch 9/15
44/44 [==============================] - 0s 3ms/step - loss: 156.0155
Epoch 10/15
44/44 [==============================] - 0s 3ms/step - loss: 150.6175
Epoch 11/15
44/44 [==============================] - 0s 3ms/step - loss: 144.5356
Epoch 12/15
44/44 [==============================] - 0s 3ms/step - loss: 143.7639
Epoch 13/15
44/44 [======

KeyboardInterrupt: 